# Tutorial 1B Hands on data - wrangling data with Python & pandas



In two parts:
* reproducing the DataWrangler process (using the same 'Air crashes' data) and 
* bad, bad data investigations


** Before you start, a most important thing to do, check your python version**

In [1]:
import sys
print (sys.version_info)

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)


In [2]:
import pandas as pd # not the bamboo eating bear... 'Panel Data' 

# Part 1 
## Step 1 Read data 

In [3]:
# load data
df = pd.read_csv('AirCrashes.csv') # df is a dataframe, confirm with: type(df)
df.shape

(927, 1)

#### How many lines of data?
#### How many did you get with DataWrangler?

Have a look a the first few rows:

In [4]:
df.head()

,Incident American Airlines Flight 11 involving a Boeing 767-223ER in 2001
Casualties,Extremely High
Total Dead,1692
Crew,11
Passengers,81
Ground,1600


That's not quite right.. the first line has been stolen for the title/header
#### Does read_csv ignore empty lines? 
e.g. line 18 should be blank

skip_blank_lines = True (the default) see:

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

Try again, force our own headers upon the data using default column names from DataWrangler (split/extract etc.) 

This pushes the first incident down into the data where it belongs (but adds a NaN, below)

In [5]:
df = pd.read_csv('AirCrashes.csv', names = ['split', 'split1']) 
# column names (split, split1) replicate DataWrangler column names

df.head(10)

,split,split1
0,Incident American Airlines Flight 11 involving...,NaN
1,Casualties,Extremely High
2,Total Dead,1692
3,Crew,11
4,Passengers,81
5,Ground,1600
6,Notes,No survivors
7,Type,INH
8,Reason,Attack
9,Location,New York - New York - US


## Step 2 Extract index(s)

### Extract flight information
The flight information is between "Incident" and "involving" in the "split" cell.
Now, extract flights based on one of the suggestions from DataWrangler, i.e., "(Incident (.*) involving)".

In [6]:
# treat the 'split' column as a str, then 
# use the extract method on the str
flight_no = df['split'].str.extract("Incident (.*) involving")

/home/zby0902/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


### That seems to have worked...

We got 'American Airlines Flight 11' and 'United Airlines Flight 175' etc but lost all the other data and gained a bunch of NaNs

#### What is this 'str.extract' code anyway? (add a comment to decode or explain it)
>**Answer** To extract substring from each cell in this roll that mathches the pattern of the regular expression
extract("Incident (.*) involving")


### But we want planes in a new column, we can create one called 'extract' like this:

df['extract'] = df['split'].str.extract("Incident (.*) involving")

But this would be the last column, we want it in the second (location is not critical but it can be done so why not). 

Now, use the DataFrame's insert function.

In [7]:
df.insert(loc=1,column='extract',value=flight_no)

In [8]:
df.head(20)

,split,extract,split1
0,Incident American Airlines Flight 11 involving...,American Airlines Flight 11,NaN
1,Casualties,NaN,Extremely High
2,Total Dead,NaN,1692
3,Crew,NaN,11
4,Passengers,NaN,81
5,Ground,NaN,1600
6,Notes,NaN,No survivors
7,Type,NaN,INH
8,Reason,NaN,Attack
9,Location,NaN,New York - New York - US


So there's the flight information in it's own column, plus a whole lot of NaNs

We could replace all the NaN with spaces or similar but they can wait

Now we want the aircraft in it's own column, similar to above, 
based on the suggestion from DataWrangler, note spaces in "\ a (.*)\ in "

#### but is this optimal?  

In [9]:
# and repeat to get the aircraft type that appears between "a" and "in"
aircraft_type = df['split'].str.extract("\ a (.*)\ in ")
df.insert(loc=2,column='extract1',value=aircraft_type)
# and df.head(20) to confirm
df.head(20)

/home/zby0902/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


,split,extract,extract1,split1
0,Incident American Airlines Flight 11 involving...,American Airlines Flight 11,Boeing 767-223ER,NaN
1,Casualties,NaN,NaN,Extremely High
2,Total Dead,NaN,NaN,1692
3,Crew,NaN,NaN,11
4,Passengers,NaN,NaN,81
5,Ground,NaN,NaN,1600
6,Notes,NaN,NaN,No survivors
7,Type,NaN,NaN,INH
8,Reason,NaN,NaN,Attack
9,Location,NaN,NaN,New York - New York - US


Drop the empty lines 

In [10]:
df['split'].duplicated();

In [11]:
df = df.dropna(how='all') 
df

,split,extract,extract1,split1
0,Incident American Airlines Flight 11 involving...,American Airlines Flight 11,Boeing 767-223ER,NaN
1,Casualties,NaN,NaN,Extremely High
2,Total Dead,NaN,NaN,1692
3,Crew,NaN,NaN,11
4,Passengers,NaN,NaN,81
5,Ground,NaN,NaN,1600
6,Notes,NaN,NaN,No survivors
7,Type,NaN,NaN,INH
8,Reason,NaN,NaN,Attack
9,Location,NaN,NaN,New York - New York - US


## Step 3  'Fill down'

We want to 'fill down' the indexes  (e.g. lines 1 to 15 should be associated with line 0)

There are several options
* na.locf() method from zoo package. 
* ddply() from plyr
* bfill()
* fillna()


In [12]:
# magic, take the previous value (not NaN) and fill down
df = df.fillna(method='pad')
# http://pandas.pydata.org/pandas-docs/stable/missing_data.html#filling-missing-values-fillna

In [13]:
df.head(200)

,split,extract,extract1,split1
0,Incident American Airlines Flight 11 involving...,American Airlines Flight 11,Boeing 767-223ER,NaN
1,Casualties,American Airlines Flight 11,Boeing 767-223ER,Extremely High
2,Total Dead,American Airlines Flight 11,Boeing 767-223ER,1692
3,Crew,American Airlines Flight 11,Boeing 767-223ER,11
4,Passengers,American Airlines Flight 11,Boeing 767-223ER,81
5,Ground,American Airlines Flight 11,Boeing 767-223ER,1600
6,Notes,American Airlines Flight 11,Boeing 767-223ER,No survivors
7,Type,American Airlines Flight 11,Boeing 767-223ER,INH
8,Reason,American Airlines Flight 11,Boeing 767-223ER,Attack
9,Location,American Airlines Flight 11,Boeing 767-223ER,New York - New York - US


## Step 4 Remove the index row

We need to delete all the 'incident' rows, they have served their purpose and are now redundant. 

In [14]:
df[df['split'].str.contains('Incident')].index

Int64Index([  0,  16,  32,  48,  64,  80,  96, 112, 128, 144, 160, 176, 192,
            208, 224, 240, 256, 272, 288, 304, 320, 336, 352, 368, 384, 400,
            416, 432, 448, 464, 480, 496, 512, 528, 544, 560, 576, 592, 608,
            624, 640, 656, 672, 688, 704, 720, 736, 752, 768, 784, 800, 816,
            832, 848, 864, 880, 896, 912],
           dtype='int64')

In [15]:
# use str.contain function to get the row index.
# keep everything that doesn't have "Incident" in it 
df = df.drop(df[df['split'].str.contains('Incident')].index)
# do we need to worry about a plane called "Incident" or "Incident weather" etc???

In [16]:
# and check with e.g. df.shape 
print(df.shape)
df

(870, 4)


,split,extract,extract1,split1
1,Casualties,American Airlines Flight 11,Boeing 767-223ER,Extremely High
2,Total Dead,American Airlines Flight 11,Boeing 767-223ER,1692
3,Crew,American Airlines Flight 11,Boeing 767-223ER,11
4,Passengers,American Airlines Flight 11,Boeing 767-223ER,81
5,Ground,American Airlines Flight 11,Boeing 767-223ER,1600
6,Notes,American Airlines Flight 11,Boeing 767-223ER,No survivors
7,Type,American Airlines Flight 11,Boeing 767-223ER,INH
8,Reason,American Airlines Flight 11,Boeing 767-223ER,Attack
9,Location,American Airlines Flight 11,Boeing 767-223ER,New York - New York - US
10,Country,American Airlines Flight 11,Boeing 767-223ER,US


## Now to 'unfold', there are several options

* melt()
* stack, unstack?
* pivot_table()
* pivot()

http://pandas.pydata.org/pandas-docs/stable/reshaping.html

In [17]:
data = df.pivot('extract', 'split', 'split1') 
# the parameters above are (left to right) index, columns, values 
# You can also write it like this:
#data = df.pivot(index = 'extract', columns = 'split', values = 'split1') 

In [18]:
data.shape 

(58, 15)

In [19]:
data.head() # 58 records, good, but lost plane type, bad
# where's 'extract1' - can we have multiple indexes or have to put that data back in?

split,Casualties,Circumstances,Country,Crew,Date,Ground,Latitude,Location,Longitude,Notes,Passengers,Phase,Reason,Total Dead,Type
extract,,,,,,,,,,,,,,,
Aeroflot Flight 217,Extremely High,Bad Visibility by Day,Russia,10,1972-10-13,0,55.755826,Moscow - Russia,37.6173,No survivors,164,APR,Accident,174,COM
Aeroflot Flight 3352,Extremely High,Bad Visibility by Night,Russia,5,1984-10-11,4,54.9666667,Omsk - Russia,73.3833333,Some survivors,169,LDG,Accident,178,COM
Aeroflot Flight 4227,Extremely High,Bad Visibility by Night,Kazakhstan,10,1890-07-08,0,43.255058,Almaty - Kazakhstan,76.912628,No survivors,156,ENR,Accident,166,COM
Aeroflot Flight 7425,Extremely High,Bad Visibility by Night,Uzbekistan,9,1985-07-10,0,42.1566667,Uchkuduk - Uzbekistan,63.5555556,No survivors,191,ENR,Accident,0,COM
Aeroflot/Moldovia (CCCP-65816),Extremely High,Bad Visibility by Night,Kazakhstan,13,1979-08-11,0,48.8125,Saykyn - Kazakhstan,46.7636111,No survivors,165,ENR,Accident,178,COM


In [20]:
df.pivot(['extract','extract1'], 'split', 'split1')

ValueError: Wrong number of items passed 870, placement implies 2

### Problem:

we have two columns we want to pivot on, 'extract' & 'extract1' (AKA flight & plane) but

pivot() can't have multiple indexes...

e.g. df.pivot(index = ['extract','extract1'], 'split', 'split1') # error

pivot_table() can but insists on doing some accounting or aggreagating too, like sum or avg, which we don't need

e.g. pd.pivot_table(df, values='split1', index = ['extract','extract1'], columns='split') # error, no function 

### Solutions? 

use pivot() then force the other column back into the data?? 

or trick pivot_table() into doing some pointless accounting (that adds up to nothing)??

try something else... stack, unstack, group, dplyr?

DIY code??



![](http://www.desktopimages.org/pictures/2014/0212/1/orig_150933.jpg)

In [21]:
# solution: make the function a copy, x = x
# data = pd.pivot_table(df, index=["extract","extract1"], columns = 'split', values = 'split1', aggfunc = lambda x: x)   
# or 
data = pd.pivot_table(df, index=["extract","extract1"], columns = 'split', values = 'split1', aggfunc = 'max') 
# ha, cop that
# http://stackoverflow.com/questions/19279229/pandas-pivot-table-with-non-numeric-values-dataerror-no-numeric-types-to-ag

data # not using df anymore, keep it as backup

,split,Casualties,Circumstances,Country,Crew,Date,Ground,Latitude,Location,Longitude,Notes,Passengers,Phase,Reason,Total Dead,Type
extract,extract1,,,,,,,,,,,,,,,
Aeroflot Flight 217,Ilyushin Il-62,Extremely High,Bad Visibility by Day,Russia,10,1972-10-13,0,55.755826,Moscow - Russia,37.6173,No survivors,164,APR,Accident,174,COM
Aeroflot Flight 3352,Tupolev Tu-154,Extremely High,Bad Visibility by Night,Russia,5,1984-10-11,4,54.9666667,Omsk - Russia,73.3833333,Some survivors,169,LDG,Accident,178,COM
Aeroflot Flight 4227,Tupolev Tu-154B-2,Extremely High,Bad Visibility by Night,Kazakhstan,10,1890-07-08,0,43.255058,Almaty - Kazakhstan,76.912628,No survivors,156,ENR,Accident,166,COM
Aeroflot Flight 7425,Tupolev Tu-154B-2,Extremely High,Bad Visibility by Night,Uzbekistan,9,1985-07-10,0,42.1566667,Uchkuduk - Uzbekistan,63.5555556,No survivors,191,ENR,Accident,0,COM
Aeroflot/Moldovia (CCCP-65816),Tupolev Tu-134A(both),Extremely High,Bad Visibility by Night,Kazakhstan,13,1979-08-11,0,48.8125,Saykyn - Kazakhstan,46.7636111,No survivors,165,ENR,Accident,178,COM
African Air (RA-26222),Antonov An-32B,Extremely High,Bad Visibility by Night,DR Congo,0,1996-01-08,237,-4.3316667,Kinshasa - DR Congo,15.3138889,Some survivors,0,ICL,Accident,37,COM
Air France Flight 447,Airbus A330-203,Extremely High,Bad Visibility by Night,Sao Pedro,12,2009-06-01,0,03.065833,Atlantic Ocean - Sao Pedro,-30.561667,No survivors,216,ENR,Accident,228,COM
Air India Flight 182,Boeing 747-237B,Extremely High,Bad Visibility by Night,Ireland,22,1985-06-23,0,51.8968917,Atlantic Ocean - Cork - Ireland,-8.4863157,No survivors,307,ENR,Attack,329,INB
Air India Flight 855,Boeing 747-237B,Extremely High,Bad Visibility by Night,India,23,1978-01-01,0,19.0759837,Arabian Sea - Mumbai - India,72.8776559,No survivors,190,ENR,Accident,213,COM


### That's most of the wrangling as was done with DataWrangler, there are a few more optional steps:
* want the manufacturer e.g. Beoing?
* remove 'extract' & 'extract1'?
* rename columns ('split') 
* extract year into new column 
* export e.g. df.to_csv(file_name, sep=',')

In [ ]:
#data.reset_index()

# Part 2 

## Wait there's more:

Bad, bad data

This data has been deliberately damaged (sorry)

Some are obvious, some are subtle (some were already there... e.g. look for 'Â')

### See if you can find them

In [ ]:
# start with a summary table
data.describe()

### describe() shows:

* count - we can see that there are 58 records across the board, no surprise (what would it mean if there were non 58s?)
* unique - looks like all the 'Casualties' are identical (unique = 1, i.e. all 'Extremely High'), maybe this column is redundant?
* top - interesting, there were two major disasters in the exact same place? Check dates?
* freq - also interesting, 3 times there were 156 passengers on flights... superstitious? Or is it bad data?

So some clues here, dig deeper:

In [ ]:
#We can also describe individual columns:
data['Crew'].describe()

In [ ]:
# nothing new here, the most common number crew size is 14 (7 times)
# what's the biggest crew?
data['Crew'].max()


Why 9 (or even '9')?

How can this max be less than 14?

Are these even numbers?


In [ ]:
data['Crew'].mean() # expecting ~15, they were all big planes?

This is weird

If they are numbers it should be higher

If they are not numbers, what does mean 'mean'? 

In [ ]:
data[data['Crew'] == 14] # look at all the crew = 14 planes, should be 7

In [ ]:
# there are none... try this:
data[data['Crew'] == '14'] 

In [ ]:
# so there's a clue, 14 vs '14' 
# what are these data types anyway?
data.dtypes

### All data are of type object... 
### Overruled:

In [ ]:
data = data.convert_objects(convert_numeric = True) 
# this is a bit brutal, can you convert when data is loaded?

In [ ]:
data.dtypes

In [ ]:
# so now we have some numbers (int & float)
data.describe()

### Better, more information, describe() now shows:
* count - as above, all 58
* mean - crew ~15, seems OK, but mean lat & long doesn't mean much... or does it?
* std - Standard Deviation
* min - now we see some problems, zero crew? Was this a way to code a hijacking, or is it missing, or should it be 10, 20, 30?
* 25, 50 & 75% are quartiles...
* max - crew 181, no way! 1692 dead, no plane is that big... or could this be Lockerbie, i.e. plane hit town

Let's investigate the crew data:

In [ ]:
data[data['Crew'] > 20] # try also e.g. < 10

So Boeing 747s have large crews...
#### 33 crew is that possible? Or is that two flights?

#### 181 crew? Same value as for 'Total dead', can you derive crew from dead minus passengers?
(or does 'Some survivors' corrupt the maths?)


# Another way to explore... plot that data:


In [ ]:
%matplotlib inline 
# notebook majik to display plots in the notebook

data[['Crew','Passengers']].plot(x= 'Crew',y= 'Passengers',kind= 'scatter');

#### And there's the extreme outlier
#### Are there any others?

In [ ]:
# so how to put a number to an outlier?
data['Crew'].max() # works, now that they are numbers

In [ ]:
data[data['Crew']==data['Crew'].max()]

## Plotting non-numeric data

In [ ]:
data['Phase'].value_counts().plot(kind='bar');
# you can guess some of these codes
# ENR = en route?
# APR = Aproach
# Takeoff, Landing
# ICL?

In [ ]:
# not much point in plotting Casualties... so what the hey
data['Casualties'].value_counts().plot(kind='bar')

# To do: find any other data problems (there are about 10)

<br>

![](http://media1.popsugar-assets.com/files/thumbor/yjoSwHRBZ4MpTO3TN6lvI_gsKMI/fit-in/2048xorig/filters:format_auto-!!-:strip_icc-!!-/2016/03/02/901/n/1922283/01f64bd801c06153_game4/i/When-Everyone-Trying-Talk-You-Youre-Too-Hungry-Care.gif)

#### Post your suspected bad data cases in Moodle discussion forums, how you found it, and suggested fixes
(one or two each, share the load)


In [ ]:
# go crazy

#### Can data be 'typed' as it is read in? 
(yes see 'dtype')
http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
#### What happens if this process encounters bad data?

#### So why is 'Crew' max '9' above?

#### When a plane hits another plane is that one record or two?

#### Can DataWrangler do this sort of wrangling? 

